# Deep Q learning: Atari

### Librerías

In [1]:
from keras.models import Sequential      
from keras.layers import Dense, Flatten  
from collections import deque            
import matplotlib.pyplot as plt
import random     
import numpy as np
import gym      
env = gym.make('Breakout-v0') 

Using TensorFlow backend.


### Modelo: Deep neural net, no usa pooling ya que no queremos reconocer ciertas carácterísticas aisladas sino que es importante su posición en la imagen. 

In [2]:
model = Sequential()
model.add(Dense(40, input_shape=(2,) + env.observation_space.shape, kernel_initializer='uniform', activation='relu'))
model.add(Flatten())       #Aplana las entradas multidimensionales
model.add(Dense(20, kernel_initializer='uniform', activation='relu'))
model.add(Dense(10, kernel_initializer='uniform', activation='relu'))
model.add(Dense(env.action_space.n, kernel_initializer='uniform', activation='softplus')) 

model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

### Parámetros del algorítmo

In [3]:
observetime = 500                          
epsilon = 0.7                              
gamma = 0.9                                
mb_size = 30                               

### Funciones útiles

In [4]:
def plotHist(history):
    plt.plot(history.history['loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train'], loc='upper left')
    plt.show()

### Entrenamiento de la red, aprende de las observaciones realizadas en el juego terminado. 

In [5]:
def trainDQN(minibatch,state):
    inputs_shape = (mb_size,) + state.shape[1:]
    inputs = np.zeros(inputs_shape)
    targets = np.zeros((mb_size, env.action_space.n))

    for i in range(0, mb_size):
        state = minibatch[i][0]
        action = minibatch[i][1]
        reward = minibatch[i][2]
        state_new = minibatch[i][3]
        done = minibatch[i][4]

        inputs[i:i+1] = np.expand_dims(state, axis=0)
        targets[i] = model.predict(state)
        Q_sa = model.predict(state_new)

        if done:
            targets[i, action] = reward
        else:
            targets[i, action] = reward + gamma * np.max(Q_sa)

        history = model.train_on_batch(inputs, targets)
        #plotHist(history)
        #if ((i%int(mb_size/3))==0): print("Learning...",i)
    #print('Fin aprendizaje')

### Observaciones y aprendizaje. 
Se guardan observaciones, los estados y sus recompensas durante varios juegos.
Con las observaciones almacenadas se entrena a la red al terminar cada juego. 

In [6]:
for i in range(0,10000):         #Cantidad de juegos con reseteo de memoria.
    #print("Nuevo comienzo de aprendizaje")
    D = deque()                                
    observation = env.reset()                    
    obs = np.expand_dims(observation, axis=0)    
    state = np.stack((obs, obs), axis=1)
    done = False
    for t in range(observetime):
        if np.random.rand() <= epsilon:
            action = np.random.randint(0, env.action_space.n, size=1)[0]
        else:
            Q = model.predict(state)          
            action = np.argmax(Q)             
        observation_new, reward, done, info = env.step(action)  
        obs_new = np.expand_dims(observation_new, axis=0)       
        state_new = np.append(np.expand_dims(obs_new, axis=0), state[:, :1, :], axis=1)  
        D.append((state, action, reward, state_new, done))     
        state = state_new   
        if done:
            minibatch = random.sample(D, mb_size)           
            trainDQN(minibatch,state)
            env.reset()           
            obs = np.expand_dims(observation, axis=0) 
            state = np.stack((obs, obs), axis=1)
    if ((i%100)==0): print("Continúa aprendizaje")

print('Fin observaciones-aprendizaje')

Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa aprendizaje
Continúa apre

### Test: Jugando con la DQN entrenada

In [7]:
observation = env.reset()

obs = np.expand_dims(observation, axis=0)
state = np.stack((obs, obs), axis=1)
done = False
tot_reward = 0.0
count = 0
while not done:
    #env.render()                    
    
    if np.random.rand() <= 0.1:
        action = np.random.randint(0, env.action_space.n, size=1)[0]
    else:
        Q = model.predict(state)     
        action = np.argmax(Q)        
    
    observation, reward, done, info = env.step(action)
    obs = np.expand_dims(observation, axis=0)
    state = np.append(np.expand_dims(obs, axis=0), state[:, :1, :], axis=1)    
    tot_reward = tot_reward + reward
    count = count + 1
    #if ((count%100)==0): print("Playing... Reward:%0.2f"%(tot_reward))
        
    if (count==500 and tot_reward==0): 
        done = True
        print("Nunca inició el juego")
        
print('Fin del juego, recompensa total: {}'.format(tot_reward))

Fin del juego, recompensa total: 41


### Bibliografía

[Becominghuman.ai - $"Lets bild an Atari"$](https://becominghuman.ai/lets-build-an-atari-ai-part-1-dqn-df57e8ff3b26)

[Neuro.cs.ut.ee](http://neuro.cs.ut.ee/demystifying-deep-reinforcement-learning/#comment-12)

[OpenAi Documentation](https://gym.openai.com/docs/)